In [ ]:
!pip install arabert transformers

In [ ]:
!pip install transformers[torch]

In [51]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import seaborn as sn
from sklearn.metrics import classification_report

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import TextClassificationPipeline

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [127]:
df=pd.read_excel('/content/drive/MyDrive/Vaccine/df_safe_train.xlsx')

In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   ID       179 non-null    float64
 1   Arabic   400 non-null    object 
 2   English  148 non-null    object 
 3   Label    399 non-null    float64
 4   text     400 non-null    object 
dtypes: float64(2), object(3)
memory usage: 15.8+ KB


In [129]:
# Encode the labels
label_encoder = LabelEncoder()
df['Label'] = label_encoder.fit_transform(df['Label'])


In [130]:
df['Label'] = df['Label'].replace(3, 2)

In [131]:
df['Label'].value_counts()

2    193
0    109
1     98
Name: Label, dtype: int64

In [132]:
# Split the data
train_texts, val_texts, train_labels, val_labels = train_test_split(df['Arabic'], df['Label'], test_size=0.2, shuffle=True)


In [133]:
# Load AraBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('aubmindlab/bert-base-arabert')
model = BertForSequenceClassification.from_pretrained('aubmindlab/bert-base-arabert', num_labels=len(label_encoder.classes_))

# Load mBERT tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('asafaya/bert-base-arabic')
#model = BertForSequenceClassification.from_pretrained('asafaya/bert-base-arabic', num_labels=len(label_encoder.classes_))

# Load BERT Large tokenizer and model
#tokenizer = BertTokenizer.from_pretrained('bert-large-cased')
#model = BertForSequenceClassification.from_pretrained('bert-large-cased', num_labels=len(df['Label'].unique()))

#Load AraElectra
#tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
#model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=len(label_encoder.classes_))



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabert and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [134]:
# Tokenize the data
train_encodings = tokenizer(train_texts.tolist(), truncation=True, padding=True)
val_encodings = tokenizer(val_texts.tolist(), truncation=True, padding=True)


In [135]:
# Prepare the dataset
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels.iloc[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)


In [136]:
from sklearn.metrics import accuracy_score
from transformers import EvalPrediction
def compute_metrics(p: EvalPrediction):
    preds = p.predictions.argmax(-1)
    return {"accuracy": (preds == p.label_ids).mean()}


In [137]:
# Training the model with accuracy as a metric
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=10,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    warmup_steps=100,
    weight_decay=0.001,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,1.222000,1.037555,0.550000
2,1.027800,1.038764,0.462500
3,1.022200,0.948768,0.550000
4,0.719700,1.159126,0.425000
5,0.663400,1.183104,0.512500
6,0.175100,1.614715,0.462500
7,0.098800,1.910102,0.537500
8,0.063100,2.160999,0.487500
9,0.041700,1.792511,0.637500
10,0.033700,2.040210,0.562500


TrainOutput(global_step=270, training_loss=0.48843288006329977, metrics={'train_runtime': 189.2112, 'train_samples_per_second': 16.912, 'train_steps_per_second': 1.427, 'total_flos': 317383409740800.0, 'train_loss': 0.48843288006329977, 'epoch': 10.0})

In [139]:
# Evaluation
results = trainer.evaluate()
print("Validation Results:", results)

Validation Results: {'eval_loss': 0.9487676620483398, 'eval_accuracy': 0.55, 'eval_runtime': 0.6648, 'eval_samples_per_second': 120.338, 'eval_steps_per_second': 10.53, 'epoch': 10.0}


In [140]:
# Calculate accuracy on validation set
val_preds = trainer.predict(val_dataset)
val_preds_labels = val_preds.predictions.argmax(-1)
accuracy = accuracy_score(val_labels, val_preds_labels)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.55


In [126]:
# Calculate precision, recall, and F1-score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

precision = precision_score(val_labels, val_preds_labels, average='weighted')
recall = recall_score(val_labels, val_preds_labels, average='weighted')
f1 = f1_score(val_labels, val_preds_labels, average='weighted')

print("Validation Precision:", precision)
print("Validation Recall:", recall)
print("Validation F1-Score:", f1)

Validation Precision: 0.5543617021276596
Validation Recall: 0.525
Validation F1-Score: 0.5158211794546608


In [ ]:
#Inference-Preidct new label

In [123]:
import pandas as pd
from torch.utils.data import DataLoader

# Assuming df_new is your new dataset with a 'text' column
df_new = pd.DataFrame({
    'text': ["This is a sample text", "Another example text", "More text samples"]
})

# Tokenize the new text data
new_encodings = tokenizer(df_new['text'].tolist(), truncation=True, padding=True, return_tensors="pt")

# Create a DataLoader for the new data
new_dataset = CustomDataset(new_encodings, pd.Series([0]*len(df_new)))  # Dummy labels
new_dataloader = DataLoader(new_dataset, batch_size=4)

# Use the trained model to make predictions
model.eval()
all_preds = []
for batch in new_dataloader:
    with torch.no_grad():
        outputs = model(**{k: v.to(model.device) for k, v in batch.items() if k != 'labels'})
        all_preds.append(outputs.logits.cpu().numpy())

# Convert output scores to predicted labels
import numpy as np
preds = np.concatenate(all_preds, axis=0)
pred_labels = preds.argmax(axis=-1)

# Map numeric labels back to original string labels
predicted_labels = label_encoder.inverse_transform(pred_labels)
print(predicted_labels)


[3. 3. 3.]


<ipython-input-117-a865eedcadf2>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
